In [61]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import re
import sys
import tarfile
import time
from six.moves import urllib
import tensorflow as tf

import cifar10_input

In [62]:
#Below code was referred to http://danijar.com/structuring-your-tensorflow-models/
import functools
def doublewrap(function):
    """
    A decorator decorator, allowing to use the decorator to be used without
    parentheses if not arguments are provided. All arguments must be optional.
    """
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator


@doublewrap
def define_scope(function, scope=None, *args, **kwargs):
    """
    A decorator for functions that define TensorFlow operations. The wrapped
    function will only be executed once. Subsequent calls to it will directly
    return the result so that operations are added to the graph only once.

    The operations added by the function live within a tf.variable_scope(). If
    this decorator is used with arguments, they will be forwarded to the
    variable scope. The scope name defaults to the name of the wrapped
    function.
    """
    attribute = '_cache_' + function.__name__#Get function name
    name = scope or function.__name__
    @property
    @functools.wraps(function)#Keep the original function
    def decorator(self):
        if not hasattr(self, attribute):#If the attribute not exist
            with tf.variable_scope(name, *args, **kwargs):#Add scope name
                setattr(self, attribute, function(self))
        return getattr(self, attribute)#otherwise return the attribute

In [72]:
#Define cifar model
TOWER_NAME = 'tower'
class cifar10_Model:
    '''Define a basic model for Cifar10 image classification, the model
    Provides graph structure of tensorflow'''
    
    def __init__(self, image_holder, label_holder, batch_size=128, num_class=10):
        self.image_holder = image_holder
        self.label_holder = label_holder
        self.num_class = num_class
        self.batch_size = batch_size
        self.prediction
        self.optimize
        self.accuracy
        print('Initializing Cifar10 Model!') 
    
    @property
    def prediction(self):
        print('Inference')
        #logits = self.inference()
        image = tf.reshape(self.image_holder, [self.batch_size, -1])
        dim = image.get_shape()[1].value
        W = tf.Variable(tf.zeros([dim, self.num_class]))
        b = tf.Variable(tf.zeros([self.num_class]))
        logits = tf.matmul(image, W) + b
        return tf.nn.softmax(logits)
        
    
    @property
    def optimize(self):
        cross_entropy = -tf.reduce_sum(self.label_holder*
                                       tf.log(self.prediction))
        optimizer = tf.train.AdamOptimizer(1e-4)
        return optimizer.minimize(cross_entropy)
    
    @property
    def accuracy(self):
        correct_prediction = tf.equal(tf.argmax(self.label_holder,1), 
                                      tf.argmax(self.prediction,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        return accuracy
    
    #create weights
    def weight_variable(self, shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)
    
    #create biases
    def bias_variable(self, shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)
    
    #@define_scope
    def inference(self):
        """Build the CIFAR-10 model.
        Args:
        images: Images returned from distorted_inputs() or inputs().
        Returns:
        Logits.
        """
        # We instantiate all variables using tf.get_variable() instead of
        # tf.Variable() in order to share variables across multiple GPU training runs.
        # If we only ran this model on a single GPU, we could simplify this function
        # by replacing all instances of tf.get_variable() with tf.Variable().
        #
        # conv1
        with tf.variable_scope('conv1') as scope:
            kernel = self._variable_with_weight_decay('weights',
                                                 shape=[5, 5, 3, 64],
                                                 stddev=5e-2,
                                                 wd=0.0)
            conv = tf.nn.conv2d(self.image_holder, kernel, [1, 1, 1, 1], padding='SAME')
            biases = self._variable_on_cpu('biases', [64], tf.constant_initializer(0.0))
            pre_activation = tf.nn.bias_add(conv, biases)
            conv1 = tf.nn.relu(pre_activation, name=scope.name)
            self._activation_summary(conv1)
            # pool1
            pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                                   padding='SAME', name='pool1')
            # norm1
            norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                              name='norm1')
        
        # conv2
        with tf.variable_scope('conv2') as scope:
            kernel = self._variable_with_weight_decay('weights',
                                                 shape=[5, 5, 64, 64],
                                                 stddev=5e-2,
                                                 wd=0.0)
            conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = self._variable_on_cpu('biases', [64], tf.constant_initializer(0.1))
            pre_activation = tf.nn.bias_add(conv, biases)
            conv2 = tf.nn.relu(pre_activation, name=scope.name)
            self._activation_summary(conv2)      
            # norm2
            norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                              name='norm2')
            # pool2
            pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1],
                                   strides=[1, 2, 2, 1], padding='SAME', name='pool2')
        
        # local3
        with tf.variable_scope('local3') as scope:
            # Move everything into depth so we can perform a single matrix multiply.
            print(pool2.shape)
            reshape_tensor = tf.reshape(pool2, [self.batch_size, -1], name='reshape_tensor')
            dim = reshape_tensor.get_shape()[1].value
            weights = self._variable_with_weight_decay('weights', shape=[dim, 384],
                                                  stddev=0.04, wd=0.004)
            biases = self._variable_on_cpu('biases', [384], tf.constant_initializer(0.1))
            local3 = tf.nn.relu(tf.matmul(reshape_tensor, weights) + biases, name=scope.name)
            self._activation_summary(local3)
        # local4
        with tf.variable_scope('local4') as scope:
            weights = self._variable_with_weight_decay('weights', shape=[384, 192],
                                                  stddev=0.04, wd=0.004)
            biases = self._variable_on_cpu('biases', [192], tf.constant_initializer(0.1))
            local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name=scope.name)
            self._activation_summary(local4)
        
        # linear layer(WX + b),
        # We don't apply softmax here because
        # tf.nn.sparse_softmax_cross_entropy_with_logits accepts the unscaled logits
        # and performs the softmax internally for efficiency.
        with tf.variable_scope('softmax_linear') as scope:
            weights = self._variable_with_weight_decay('weights', [192, self.num_class],
                                                  stddev=1/192.0, wd=0.0)
            biases = self._variable_on_cpu('biases', [self.num_class],
                                      tf.constant_initializer(0.0))
            softmax_linear = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
            self._activation_summary(softmax_linear)
        
        return softmax_linear
    
    def _activation_summary(self, x):
        """Helper to create summaries for activations.
        Creates a summary that provides a histogram of activations.
        Creates a summary that measures the sparsity of activations.
        Args:
        x: Tensor
        Returns:
        nothing
        """
        # Remove 'tower_[0-9]/' from the name in case this is a multi-GPU training
        # session. This helps the clarity of presentation on tensorboard.
        tensor_name = re.sub('%s_[0-9]*/' % TOWER_NAME, '', x.op.name)
        tf.summary.histogram(tensor_name + '/activations', x)
        tf.summary.scalar(tensor_name + '/sparsity',
                          tf.nn.zero_fraction(x))
        
    def _variable_on_cpu(self, name, shape, initializer):
        """Helper to create a Variable stored on CPU memory.
        Args:
        name: name of the variable
        shape: list of ints
        initializer: initializer for Variable
        Returns:
        Variable Tensor
        """
        with tf.device('/cpu:0'):
            dtype = tf.float32
            var = tf.Variable(initializer(shape), dtype=dtype, name=name)
        return var

    def _variable_with_weight_decay(self, name, shape, stddev, wd):
        """Helper to create an initialized Variable with weight decay.
        Note that the Variable is initialized with a truncated normal distribution.
        A weight decay is added only if one is specified.
        Args:
        name: name of the variable
        shape: list of ints
        stddev: standard deviation of a truncated Gaussian
        wd: add L2Loss weight decay multiplied by this float. If None, weight
        decay is not added for this Variable.
        Returns:
        Variable Tensor
        """
        dtype = tf.float32
        var = self._variable_on_cpu(name,
                               shape,
                               tf.truncated_normal_initializer(stddev=stddev, dtype=dtype))
        if wd is not None:
            weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
            tf.add_to_collection('losses', weight_decay)
        return var

In [73]:
def train_test(model, num_steps, data_dir, dropout=False):
    #训练测试函数
    train_images, train_labels = cifar10_input.distorted_inputs(data_dir=data_dir,
                                                                batch_size=128)
    test_images, test_labels = cifar10_input.inputs(eval_data=True,
                                                    data_dir=data_dir,
                                                    batch_size=128)
    sess = tf.Session()
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    #tf.train.start_queue_runners()
    print('Start Queue!')
    for step in range(num_steps):
        #, batch_labels = mnist.train.next_batch(batch_size)
        print('Feed distorted input dataset!')
        batch_data, batch_labels = sess.run([train_images, train_labels])
        
        #将数据传递给字典
        feed_dict = {model.image_holder : batch_data, 
                     model.label_holder: batch_labels}
       # if dropout:
            #feed_dict[model.keep_prob] = 0.8
        _, acc = sess.run([model.optimize, model.accuracy], feed_dict=feed_dict)
        print('Training accuracy:', acc)
        #_ = sess.run(model.optimize)
        if step%2 == 0:
            #定义模型评价指标精确度
            batch_data, batch_labels = sess.run([test_images, test_labels])
            feed_dict={model.image_holder: batch_data, 
                       model.label_holder: batch_labels}
            #if dropout:
                #feed_dict[model.keep_prob] = 1
            accuracy = sess.run(model.accuracy, feed_dict=feed_dict)
            #accuracy = sess.run(model.accuracy)
            print("测试精度：", accuracy)

In [74]:
batch_size = 128
data_dir = "cifar10_data/cifar-10-batches-bin"
num_steps = 101
image_holder = tf.placeholder(tf.float32, [batch_size, 24, 24, 3])
label_holder = tf.placeholder(tf.float32, [batch_size])

In [75]:
model = cifar10_Model(image_holder, label_holder, cifar10_input.NUM_CLASSES)

Inference


ValueError: Dimension size must be evenly divisible by 10 but is 221184 for 'Reshape_3' (op: 'Reshape') with input shapes: [128,24,24,3], [2].

In [67]:
model.accuracy

In [ ]:
train_test(model, num_steps, data_dir, dropout=False)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
Start Queue!
Feed distorted input dataset!


In [43]:
import numpy as np
pool2 = np.random.randn(128, 6, 6, 64)
tf.reshape(pool2, [128, -1])

<tf.Tensor 'Reshape:0' shape=(128, 2304) dtype=float64>

In [95]:
6*6*6*4*128*2

221184

In [76]:
image = tf.reshape(image_holder, [batch_size, -1])

In [77]:
image

<tf.Tensor 'Reshape_4:0' shape=(128, 1728) dtype=float32>

In [87]:
batch_size = 128
data_dir = "cifar10_data/cifar-10-batches-bin"
num_steps = 101
num_class = 10
image_holder = tf.placeholder(tf.float32, [batch_size, 24, 24, 3])
label_holder = tf.placeholder(tf.float32, [batch_size])

In [88]:
graph = tf.Graph()
with graph.as_default():
    image_holder = tf.placeholder(tf.float32, [batch_size, 24, 24, 3])
    label_holder = tf.placeholder(tf.float32, [batch_size])
    def inference(image):
        """Build the CIFAR-10 model.
        Args:
        images: Images returned from distorted_inputs() or inputs().
        Returns:
        Logits.
        """
        # We instantiate all variables using tf.get_variable() instead of
        # tf.Variable() in order to share variables across multiple GPU training runs.
        # If we only ran this model on a single GPU, we could simplify this function
        # by replacing all instances of tf.get_variable() with tf.Variable().
        #
        # conv1
        with tf.variable_scope('conv1') as scope:
            kernel = _variable_with_weight_decay('weights',
                                                 shape=[5, 5, 3, 64],
                                                 stddev=5e-2,
                                                 wd=0.0)
            conv = tf.nn.conv2d(image, kernel, [1, 1, 1, 1], padding='SAME')
            biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.0))
            pre_activation = tf.nn.bias_add(conv, biases)
            conv1 = tf.nn.relu(pre_activation, name=scope.name)
            _activation_summary(conv1)
            # pool1
            pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                                   padding='SAME', name='pool1')
            # norm1
            norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                              name='norm1')
        
        # conv2
        with tf.variable_scope('conv2') as scope:
            kernel = _variable_with_weight_decay('weights',
                                                 shape=[5, 5, 64, 64],
                                                 stddev=5e-2,
                                                 wd=0.0)
            conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.1))
            pre_activation = tf.nn.bias_add(conv, biases)
            conv2 = tf.nn.relu(pre_activation, name=scope.name)
            _activation_summary(conv2)      
            # norm2
            norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                              name='norm2')
            # pool2
            pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1],
                                   strides=[1, 2, 2, 1], padding='SAME', name='pool2')
        
        # local3
        with tf.variable_scope('local3') as scope:
            # Move everything into depth so we can perform a single matrix multiply.
            print(pool2.shape)
            reshape_tensor = tf.reshape(pool2, [batch_size, -1], name='reshape_tensor')
            dim = reshape_tensor.get_shape()[1].value
            weights = _variable_with_weight_decay('weights', shape=[dim, 384],
                                                  stddev=0.04, wd=0.004)
            biases = _variable_on_cpu('biases', [384], tf.constant_initializer(0.1))
            local3 = tf.nn.relu(tf.matmul(reshape_tensor, weights) + biases, name=scope.name)
            _activation_summary(local3)
        # local4
        with tf.variable_scope('local4') as scope:
            weights = _variable_with_weight_decay('weights', shape=[384, 192],
                                                  stddev=0.04, wd=0.004)
            biases = _variable_on_cpu('biases', [192], tf.constant_initializer(0.1))
            local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name=scope.name)
            _activation_summary(local4)
        
        # linear layer(WX + b),
        # We don't apply softmax here because
        # tf.nn.sparse_softmax_cross_entropy_with_logits accepts the unscaled logits
        # and performs the softmax internally for efficiency.
        with tf.variable_scope('softmax_linear') as scope:
            weights = _variable_with_weight_decay('weights', [192, num_class],
                                                  stddev=1/192.0, wd=0.0)
            biases = _variable_on_cpu('biases', [self.num_class],
                                      tf.constant_initializer(0.0))
            softmax_linear = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
            _activation_summary(softmax_linear)
        
        return softmax_linear
    
    def _activation_summary(x):
        """Helper to create summaries for activations.
        Creates a summary that provides a histogram of activations.
        Creates a summary that measures the sparsity of activations.
        Args:
        x: Tensor
        Returns:
        nothing
        """
        # Remove 'tower_[0-9]/' from the name in case this is a multi-GPU training
        # session. This helps the clarity of presentation on tensorboard.
        tensor_name = re.sub('%s_[0-9]*/' % TOWER_NAME, '', x.op.name)
        tf.summary.histogram(tensor_name + '/activations', x)
        tf.summary.scalar(tensor_name + '/sparsity',
                          tf.nn.zero_fraction(x))
        
    def _variable_on_cpu(name, shape, initializer):
        """Helper to create a Variable stored on CPU memory.
        Args:
        name: name of the variable
        shape: list of ints
        initializer: initializer for Variable
        Returns:
        Variable Tensor
        """
        with tf.device('/cpu:0'):
            dtype = tf.float32
            var = tf.Variable(initializer(shape), dtype=dtype, name=name)
        return var

    def _variable_with_weight_decay(name, shape, stddev, wd):
        """Helper to create an initialized Variable with weight decay.
        Note that the Variable is initialized with a truncated normal distribution.
        A weight decay is added only if one is specified.
        Args:
        name: name of the variable
        shape: list of ints
        stddev: standard deviation of a truncated Gaussian
        wd: add L2Loss weight decay multiplied by this float. If None, weight
        decay is not added for this Variable.
        Returns:
        Variable Tensor
        """
        dtype = tf.float32
        var = _variable_on_cpu(name,
                               shape,
                               tf.truncated_normal_initializer(stddev=stddev, dtype=dtype))
        if wd is not None:
            weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
            tf.add_to_collection('losses', weight_decay)
        return var

In [89]:
with graph.as_default():
    logits = inference(image_holder)

(128, 6, 6, 64)


NameError: name 'self' is not defined